In [ ]:
import json

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
def create_driver():
    op = webdriver.ChromeOptions()
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    op.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Chrome(executable_path='./chromedriver',options=op)
    return driver

In [ ]:
def scrape_songs(url, output_file, sleep_time=0):
    driver = create_driver()
    
    driver.get(url)
    
    songs = dict()
    element = driver.find_element_by_id('listAlbum')

    div_list = element.find_elements_by_xpath('div')

    curr_album = None
    for div in div_list:
        try:
            if div.get_attribute('id'):
                album_name = div.find_element_by_xpath('b').text.replace('"','')
                songs.update({album_name: {
                    'year': int(div.text[div.text.find("(")+1:div.text.find(")")]),
                    'songs': []
                }})
                curr_album = album_name
            else:
                song_elem = div.find_element_by_xpath('a')
                song_dict = {
                    'title': song_elem.text,
                    'url': song_elem.get_attribute('href')
                }
                songs[album_name]['songs'].append(song_dict)
        except Exception as e:
            pass
    
    for album in songs.keys():
        for song_index in range(len(songs[album]['songs'])):
            
            url = songs[album]['songs'][song_index]['url']

            driver.get(url)

            lyrics_xpath = '/html/body/div[2]/div/div[2]/div[5]'
            element = driver.find_element_by_xpath(lyrics_xpath)
            songs[album]['songs'][song_index]['lyrics'] = element.text
            
            time.sleep(sleep_time)
            
    with open(output_file, 'w') as fp:
        json.dump(songs, fp, indent=4)
        
    return songs

In [ ]:
scrape_songs('https://www.azlyrics.com/m/mychemicalromance.html', output_file='mcr_sample_data.json')